## 3. Path Planning

The goal of this section is to find the shortest path between two points in a grid-like structure while minimizing the total cost associated with the path. <br/>
We chose to use the A* star Algorithm because it it garanteed to find the shortest path and it is more efficient than other algorithms in terms of both time and memory usage.<br/>
We use this algorithm to find the shortest path from the robot initial position to the robot targeted position. At this point of the project we are taking into account only the global obstacles visible from the camera, and not the local ones.